In [1]:
import cv2
import numpy as np

def label_connected_components(binary_image):
    height, width = binary_image.shape
    labels = np.zeros((height, width), dtype=np.int32)
    current_label = 1
    equivalences = {}

    def find_root(label):
        root = label
        while root in equivalences:
            root = equivalences[root]
        return root

    component_count = 0

    for y in range(height):
        for x in range(width):
            if binary_image[y, x] == 1:
                neighbors = [0, 0, 0, 0]

                if x > 0:
                    neighbors[0] = labels[y, x - 1]
                if y > 0:
                    neighbors[1] = labels[y - 1, x]
                if x > 0 and y > 0:
                    neighbors[2] = labels[y - 1, x - 1]
                if x < width - 1 and y > 0:
                    neighbors[3] = labels[y - 1, x + 1]

                neighbor_labels = [label for label in neighbors if label != 0]

                if not neighbor_labels:
                    labels[y, x] = current_label
                    component_count += 1
                    current_label += 1
                else:
                    min_label = min(neighbor_labels)
                    labels[y, x] = min_label

                    for label in neighbor_labels:
                        if label != min_label:
                            equivalences[label] = min_label

    return labels, component_count

In [2]:
# Usage Example:
binary_image = cv2.imread('sample.jpeg', cv2.IMREAD_GRAYSCALE)
binary_image[binary_image < 128] = 0
binary_image[binary_image >= 128] = 1

connected_components, blob_count = label_connected_components(binary_image)

# Make Fancy Color #
colors = np.random.randint(0, 255, size=(blob_count, 3), dtype=np.uint8)
colored_image = np.zeros((binary_image.shape[0], binary_image.shape[1], 3), dtype=np.uint8)

for label in range(1, blob_count + 1):
    mask = connected_components == label
    colored_image[mask] = colors[label - 1]
####################

print(f"Blob Count is: {blob_count}")

cv2.imshow('Colored Connected Components', colored_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

Blob Count is: 175
